In [1]:
import pyspark
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf, split, col, regexp_extract,regexp_replace
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml import Pipeline
from pyspark.ml.feature import  NGram, Tokenizer,CountVectorizer, StopWordsRemover
import pandas as pd 
from functools import reduce
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
dataset = spark.read.csv("/home/gifty/Documents/gsg_test/question1_data.csv", header=True, inferSchema=True)

In [4]:
dataset.select('VF_DATA').collect()

[Row(VF_DATA='AGE=28;SEX=M;CF=0;REGNUM=3;BEST_PHONE=land;TSCORE=80;PSCORE=56;G2016=M;DENSITY=3;COUNTY=SPOKANE;AGENTNUM=4523'),
 Row(VF_DATA='AGE=27;SEX=M;CF=0;REGNUM=1;BEST_PHONE=land;TSCORE=25;PSCORE=43;G2016=;DENSITY=5;COUNTY=SPOKANE;AGENTNUM=4418'),
 Row(VF_DATA='AGE=71;SEX=F;CF=0;REGNUM=1;BEST_PHONE=land;TSCORE=80;PSCORE=58;G2016=M;DENSITY=5;COUNTY=SPOKANE;AGENTNUM=4426'),
 Row(VF_DATA='AGE=67;SEX=M;CF=0;REGNUM=1;BEST_PHONE=land;TSCORE=97;PSCORE=97;G2016=M;DENSITY=4;COUNTY=SPOKANE;AGENTNUM=3682'),
 Row(VF_DATA='AGE=66;SEX=M;CF=0;REGNUM=2;BEST_PHONE=land;TSCORE=96;PSCORE=21;G2016=M;DENSITY=2;COUNTY=SPOKANE;AGENTNUM=4134'),
 Row(VF_DATA='AGE=82;SEX=M;CF=0;REGNUM=3;BEST_PHONE=land;TSCORE=81;PSCORE=28;G2016=M;DENSITY=2;COUNTY=SPOKANE;AGENTNUM=4418'),
 Row(VF_DATA='AGE=72;SEX=M;CF=0;REGNUM=1;BEST_PHONE=land;TSCORE=96;PSCORE=98;G2016=M;DENSITY=5;COUNTY=SPOKANE;AGENTNUM=4611'),
 Row(VF_DATA='AGE=76;SEX=M;CF=0;REGNUM=2;BEST_PHONE=land;TSCORE=96;PSCORE=2;G2016=M;DENSITY=4;COUNTY=SPOKANE;AGE

In [5]:
dataset = dataset\
            .withColumn('Age', F.split(dataset['VF_DATA'], ';')[0])\
            .withColumn('Sex', F.split(dataset['VF_DATA'], ';')[1])\
            .withColumn('TScore', F.split(dataset['VF_DATA'], ';')[5])   

In [6]:
dataset.columns

['RESPID',
 'StartDate',
 'StartTime',
 'EndDate',
 'EndTime',
 'VER',
 'VER2',
 'VER3',
 'VER4',
 'QAREA',
 'S1',
 'S2',
 'D100',
 'D101',
 'Q1',
 'P1X',
 'P1Y',
 'A2',
 'Q3',
 'Q4',
 'Q5',
 'Q6',
 'Q7',
 'A8',
 'B8',
 'Q9',
 'A10',
 'B10',
 'V1',
 'B11',
 'Q12',
 'A13',
 'B13',
 'V2',
 'E14',
 'Q15',
 'Q16',
 'Q17',
 'Q18',
 'V3',
 'Q19',
 'V4',
 'Q20',
 'G21',
 'H21',
 'G22',
 'H22',
 'Q23',
 'Q24',
 'Q25',
 'Q26',
 'V5',
 'Q27',
 'D102',
 'D300',
 'D301',
 'D105',
 'D130',
 'Q28',
 'D106',
 'D112',
 'VF_DATA',
 'Age',
 'Sex',
 'TScore']

In [7]:
dataset = dataset.drop('VF_DATA')

In [8]:
dataset.show()

+------+---------+---------+--------+-------+---+----+----+----+-----+---+---+----+----+---+---+----+----+---+---+---+---+---+----+----+---+----+----+---+----+---+----+----+---+----+---+---+---+---+---+---+---+---+----+----+----+----+---+---+---+---+---+---+----+----+----+----+----+-------+----+----+------+-----+---------+
|RESPID|StartDate|StartTime| EndDate|EndTime|VER|VER2|VER3|VER4|QAREA| S1| S2|D100|D101| Q1|P1X| P1Y|  A2| Q3| Q4| Q5| Q6| Q7|  A8|  B8| Q9| A10| B10| V1| B11|Q12| A13| B13| V2| E14|Q15|Q16|Q17|Q18| V3|Q19| V4|Q20| G21| H21| G22| H22|Q23|Q24|Q25|Q26| V5|Q27|D102|D300|D301|D105|D130|    Q28|D106|D112|   Age|  Sex|   TScore|
+------+---------+---------+--------+-------+---+----+----+----+-----+---+---+----+----+---+---+----+----+---+---+---+---+---+----+----+---+----+----+---+----+---+----+----+---+----+---+---+---+---+---+---+---+---+----+----+----+----+---+---+---+---+---+---+----+----+----+----+----+-------+----+----+------+-----+---------+
|110000| 7/9/2018|  4:29 

In [9]:
new_dataset = dataset

In [10]:
new_dataset.show()

+------+---------+---------+--------+-------+---+----+----+----+-----+---+---+----+----+---+---+----+----+---+---+---+---+---+----+----+---+----+----+---+----+---+----+----+---+----+---+---+---+---+---+---+---+---+----+----+----+----+---+---+---+---+---+---+----+----+----+----+----+-------+----+----+------+-----+---------+
|RESPID|StartDate|StartTime| EndDate|EndTime|VER|VER2|VER3|VER4|QAREA| S1| S2|D100|D101| Q1|P1X| P1Y|  A2| Q3| Q4| Q5| Q6| Q7|  A8|  B8| Q9| A10| B10| V1| B11|Q12| A13| B13| V2| E14|Q15|Q16|Q17|Q18| V3|Q19| V4|Q20| G21| H21| G22| H22|Q23|Q24|Q25|Q26| V5|Q27|D102|D300|D301|D105|D130|    Q28|D106|D112|   Age|  Sex|   TScore|
+------+---------+---------+--------+-------+---+----+----+----+-----+---+---+----+----+---+---+----+----+---+---+---+---+---+----+----+---+----+----+---+----+---+----+----+---+----+---+---+---+---+---+---+---+---+----+----+----+----+---+---+---+---+---+---+----+----+----+----+----+-------+----+----+------+-----+---------+
|110000| 7/9/2018|  4:29 

In [11]:
new_dataset = new_dataset\
            .withColumn('Age', F.translate(F.col('Age'), 'AGE=', '').alias('Age1'))\
            .withColumn('Sex', F.translate(F.col('Sex'), 'SEX=', '').alias('Sex1'))\
            .withColumn('TScore', F.translate(F.col('TScore'), 'TSCORE=', '').alias('TScore1'))

In [12]:
new_dataset.columns

['RESPID',
 'StartDate',
 'StartTime',
 'EndDate',
 'EndTime',
 'VER',
 'VER2',
 'VER3',
 'VER4',
 'QAREA',
 'S1',
 'S2',
 'D100',
 'D101',
 'Q1',
 'P1X',
 'P1Y',
 'A2',
 'Q3',
 'Q4',
 'Q5',
 'Q6',
 'Q7',
 'A8',
 'B8',
 'Q9',
 'A10',
 'B10',
 'V1',
 'B11',
 'Q12',
 'A13',
 'B13',
 'V2',
 'E14',
 'Q15',
 'Q16',
 'Q17',
 'Q18',
 'V3',
 'Q19',
 'V4',
 'Q20',
 'G21',
 'H21',
 'G22',
 'H22',
 'Q23',
 'Q24',
 'Q25',
 'Q26',
 'V5',
 'Q27',
 'D102',
 'D300',
 'D301',
 'D105',
 'D130',
 'Q28',
 'D106',
 'D112',
 'Age',
 'Sex',
 'TScore']

In [14]:
new_dataset.write.csv('question1a.csv')